In [1]:
from UTILITY_quickstart import *

In [2]:
importedDefaultSettings = loadConfig("setLattice_configs/2024-10-14_twoBunch.yml")

In [3]:
csrTF = True
evalElement = "PENT"
#evalElement = "BEGPDC"


inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"]
bunchCount = importedDefaultSettings["bunchCount"]
tao = initializeTao(
    inputBeamFilePathSuffix = inputBeamFilePathSuffix,
    
    csrTF = csrTF,
    numMacroParticles=1e5,
    scratchPath = "/tmp",
    randomizeFileNames = True
)

setLattice(tao, **importedDefaultSettings)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR on
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
Number of macro particles = 100000.0
Beam created, written to /tmp/beams/activeBeamFile_15353809695049324378.h5, and reinit to tao


In [4]:
def getBeamVectors(P):
    PDrive, PWitness = getDriverAndWitness(P) 
    return [ 
        np.median(PDrive.x),
        np.median(PDrive.xp),
        np.median(PDrive.y),
        np.median(PDrive.yp),
        np.median(PWitness.x),
        np.median(PWitness.xp),
        np.median(PWitness.y),
        np.median(PWitness.yp)
    ]
    

## Consider the original, unmodified config

In [5]:
trackBeam(tao, 
          **importedDefaultSettings,
          verbose = True
         )

P = getBeamAtElement(tao, "PENT")
getBeamVectors(P)

Loaded /tmp/beams/activeBeamFile_15353809695049324378.h5
Set track_start = L0AFEND, track_end = end
Tracking!
trackBeam() exiting


[7.311800080222429e-06,
 -4.007904214951678e-05,
 1.2451340872544295e-05,
 -8.575936991471964e-05,
 -2.00336652800646e-05,
 -3.423368241451779e-06,
 7.941285863693396e-06,
 -0.00018190683957616582]

In [6]:
PStart = getBeamAtElement( tao, "BEGBC20" ) 

makeBeamActiveBeamFile(PStart, tao = tao)

trackBeam(tao, trackStart = "BEGBC20", **importedDefaultSettings)

P = getBeamAtElement(tao, "PENT")
getBeamVectors(P)

[7.2817997957724946e-06,
 -4.0037595779413796e-05,
 1.2452308419538527e-05,
 -8.576095658350958e-05,
 -2.002919565089196e-05,
 -3.4321163039570728e-06,
 7.937097140298574e-06,
 -0.0001819162303863932]

## Baseline, sextupoles centered

In [7]:
centerSextupolesDict = {
    "S1EL_xOffset" : 0.0,
    "S1EL_yOffset" : 0.0,
    "S2EL_xOffset" : 0.0,
    "S2EL_yOffset" : 0.0,
    "S2ER_xOffset" : 0.0,
    "S2ER_yOffset" : 0.0,
    "S1ER_xOffset" : 0.0,
    "S1ER_yOffset" : 0.0,   
}

setLattice(tao, **( importedDefaultSettings | centerSextupolesDict) )

trackBeam(tao, trackStart = "BEGBC20", **importedDefaultSettings)

P = getBeamAtElement(tao, "PENT")

baselineVectors = getBeamVectors(P)
baselineVectors

resultsForExport = [ baselineVectors ] 

## Perturb each mover

In [8]:
for key in centerSextupolesDict.keys():
    setLattice(tao, **( importedDefaultSettings | centerSextupolesDict | {key : 100e-6} ) )

    trackBeam(tao, trackStart = "BEGBC20", **importedDefaultSettings)
    
    P = getBeamAtElement(tao, "PENT")

    newVectors = getBeamVectors(P)

    resultsForExport.append(newVectors)
    print( newVectors )

[-6.959261295667714e-05, -6.637147425620735e-05, -5.371072357527123e-05, -6.214012436546557e-05, -0.0002383997781339838, -0.0003047855563798381, -6.986990999762629e-05, -6.895671204035786e-05]
[-7.919009504757702e-05, -5.9690994414452305e-05, -5.8214193821997537e-05, -8.790122404263922e-05, -0.00026958464855350226, -0.00027479885988552805, -9.211044380960166e-05, -0.0001270020424979209]
[-0.00010013417233623923, -4.9531574413632115e-05, -5.364545361687565e-05, -6.235175520818027e-05, -0.00032698007699453937, -0.00022392688530309466, -6.994071194154933e-05, -6.895354695432786e-05]
[-7.95586913525356e-05, -5.9588796242747774e-05, -5.046442157882798e-05, -4.2496701298658015e-05, -0.00026992712223708286, -0.00027446670548440945, -5.277025313361071e-05, -1.3299523898695042e-05]
[-6.845577401288916e-05, -8.18678002648117e-05, -5.369922580176271e-05, -6.229235939898596e-05, -0.00024985660927741136, -0.0003373889803286598, -6.989658068436435e-05, -6.896620043171099e-05]
[-7.880545155811606e-05

In [9]:
resultsForExport = np.array(resultsForExport)
np.savetxt("/Users/nmajik/vectors.csv", resultsForExport, delimiter=',')